# 01 - Pré-processamento

Pipeline padronizado de preparação dos dados.

In [6]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
import os
from pathlib import Path
import glob
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer

# 1 ABERTURA DOS DADOS

In [7]:
data_dir = "C:/Users/Elaine/Desktop/TCC_acidentes/data/" 

In [8]:
df = pd.read_csv(os.path.join(data_dir, "tratados/amostra_19-21.csv"), sep= ',', encoding="ISO-8859-1")
df.head()

,uf,br,km,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,ano,frota,gravidade
0,RO,364.0,12.0,sexta-feira,Pleno dia,Decrescente,CÃ©u Claro,Simples,InterseÃ§Ã£o de Vias,Sim,Motoneta,2021,71697.0,0
1,MA,10.0,332.0,sexta-feira,Pleno dia,Decrescente,CÃ©u Claro,Simples,Reta,Sim,UtilitÃ¡rio,2021,47507.0,0
2,PI,343.0,342.0,segunda-feira,Pleno dia,Crescente,Nublado,MÃºltipla,RotatÃ³ria,Sim,CaminhÃ£o-trator,2020,524976.0,0
3,BA,116.0,419.0,sexta-feira,Anoitecer,Decrescente,Nublado,Dupla,Reta,Sim,Motocicleta,2020,303018.0,0
4,SP,116.0,211.0,sexta-feira,Plena Noite,Decrescente,Nublado,MÃºltipla,Reta,Sim,Motocicleta,2021,743095.0,0


# 2 TRAIN TEST SPLIT

In [9]:

X = df.drop('gravidade', axis=1)
y = df['gravidade']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.info()
X_train.head()

<class 'pandas.core.frame.DataFrame'>
Index: 91628 entries, 44026 to 43353
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uf                      91628 non-null  object 
 1   br                      91433 non-null  float64
 2   km                      91433 non-null  float64
 3   dia_semana              91628 non-null  object 
 4   fase_dia                91628 non-null  object 
 5   sentido_via             91628 non-null  object 
 6   condicao_metereologica  91628 non-null  object 
 7   tipo_pista              91628 non-null  object 
 8   tracado_via             91628 non-null  object 
 9   uso_solo                91628 non-null  object 
 10  tipo_veiculo            88174 non-null  object 
 11  ano                     91628 non-null  int64  
 12  frota                   90614 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 9.8+ MB


,uf,br,km,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,ano,frota
44026,GO,40.0,16.0,sÃ¡bado,Plena Noite,Crescente,CÃ©u Claro,Dupla,Reta,Sim,AutomÃ³vel,2019,66560.0
9134,PE,101.0,81.0,terÃ§a-feira,Pleno dia,Crescente,CÃ©u Claro,MÃºltipla,Retorno Regulamentado,NÃ£o,Semireboque,2021,217949.0
54794,PR,476.0,135.0,quarta-feira,Pleno dia,Decrescente,CÃ©u Claro,MÃºltipla,Reta,Sim,AutomÃ³vel,2020,1622891.0
106025,SC,101.0,311.0,quarta-feira,Plena Noite,Crescente,Nublado,Simples,RotatÃ³ria,NÃ£o,AutomÃ³vel,2021,31357.0
52102,MS,163.0,490.0,quinta-feira,Plena Noite,Crescente,CÃ©u Claro,Simples,RotatÃ³ria,Sim,Ãnibus,2019,613060.0


# 3 PROCESSORS

In [10]:
numeric_features = ['frota']
categorical_features = ['br',
    'ano',
    'uf', 'km', 'dia_semana', 'fase_dia', #! municipio retirado
    'sentido_via', 'condicao_metereologica', 'tipo_pista',
    'tracado_via', 'uso_solo', 'tipo_veiculo'
]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', 
                             min_frequency=0.001 #! reduzir mais as colunas
                             )
                             )
])

preprocess = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ], 
    n_jobs= 2
)

In [11]:
preprocess.fit_transform(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1191164 stored elements and shape (91628, 578)>

# 4 EXPORT

In [12]:
# Salvar o pipeline de pré-processamento
joblib.dump(preprocess, 'preprocess.pkl')
#joblib.dump(preprocess_catboost, 'preprocess_catboost.pkl')

# (opcional, mas recomendado) salvar os splits
joblib.dump(
    (X_train, X_test, y_train, y_test),
    'data_splits.pkl'
)

print("Pré-processamento e splits salvos com sucesso.")


Pré-processamento e splits salvos com sucesso.
